In [ ]:
# ===== ЭТО МОДУЛИ установленные по умолчанию при создании проекта =====

from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.gdino import GroundingDINO
from grid.model.perception.detection.rt_detr import RT_DETR
from grid.model.perception.segmentation.sam2 import SAM2
from grid.model.perception.segmentation.gsam import GroundedSAM
from grid.model.perception.depth.midas import MIDAS
from grid.model.perception.depth.zoedepth import ZoeDepth

nav_inspect_0 = ObjectInspect()
nav_visualservoing_0 = VisualServoing()
detection_gdino_0 = GroundingDINO()
detection_rt_detr_0 = RT_DETR()
seg_sam2_0 = SAM2()
seg_gsam_0 = GroundedSAM()
depth_midas_0 = MIDAS()
depth_zoedepth_0 = ZoeDepth()

In [ ]:
# ===== Импорты для текущей работы =====

import airgen
import numpy as np
import rerun as rr
import time
from math import cos, sin, pi


In [ ]:
# ===== Подключени к дрону взлет вертикальный =====

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)

# Взлёт
client.takeoffAsync().join()


In [ ]:
# ===== GPS =====
gps_data = client.getGpsData()

lat = gps_data.gnss.geo_point.latitude
lon = gps_data.gnss.geo_point.longitude
alt = gps_data.gnss.geo_point.altitude

print(f"GPS координаты: широта = {lat}, долгота = {lon}, высота = {alt}")


In [ ]:
# ===== Получени данных Lidar + визуализация =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))

In [ ]:
# ===== Построение и визуализация кругового маршрута =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))

# Стартовая точка
start = client.getMultirotorState().kinematics_estimated.position
takeoff_alt = -15  # минус = вверх в NED
center = start
radius = 10
steps = 12  # количество точек круга

# Построение круга
trajectory = []
for i in range(steps):
    angle = 2 * pi * (i / steps)
    x = center.x_val + radius * cos(angle)
    y = center.y_val + radius * sin(angle)
    z = takeoff_alt
    trajectory.append(airgen.Vector3r(x, y, z))

# Возврат к центру
trajectory.append(airgen.Vector3r(center.x_val, center.y_val, takeoff_alt))

# Визуализация маршрута
rr.log("telemetry/path", rr.LineStrips3D([[(p.x_val, p.y_val, p.z_val) for p in trajectory]]))


In [ ]:

# ===== Плавное движение по маршруту и посадка =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))
velocity = 3.0  # Плавная скорость

client.moveOnPathAsync(
    trajectory,
    velocity,
    300,
    airgen.DrivetrainType.ForwardOnly,
    airgen.YawMode(False, 0),
    -1, 0
).join()

# Плавная посадка
landing_point = airgen.Vector3r(center.x_val, center.y_val, -2)
client.moveToPositionAsync(landing_point.x_val, landing_point.y_val, landing_point.z_val, 1).join()
client.landAsync().join()
